Based on https://github.com/ivanovitchm/network_analysis/blob/main/week_08/Twitter.ipynb

In [48]:
!pip install TwitterAPI requests

In [159]:
from TwitterAPI import TwitterAPI
import networkx as nx
import requests
import time

In [10]:
# open the keys file
my_file = open("keys.txt", "r")

# read the raw data
content = my_file.read()

# split all lines by  newline character
API_KEY, API_SECRET_KEY = content.split("\n")

# close the file
my_file.close()

In [11]:
api = TwitterAPI(API_KEY, API_SECRET_KEY, auth_type='oAuth2')

In [17]:
#Get available trendingst
available_trends = api.request('trends/available').json()

In [18]:
#Filter the trends
filtered_available_trends = [i for i in available_trends if i['countryCode']=='BR']
filtered_available_trends

[{'country': 'Brazil',
  'countryCode': 'BR',
  'name': 'Brasília',
  'parentid': 23424768,
  'placeType': {'code': 7, 'name': 'Town'},
  'url': 'http://where.yahooapis.com/v1/place/455819',
  'woeid': 455819},
 {'country': 'Brazil',
  'countryCode': 'BR',
  'name': 'Belém',
  'parentid': 23424768,
  'placeType': {'code': 7, 'name': 'Town'},
  'url': 'http://where.yahooapis.com/v1/place/455820',
  'woeid': 455820},
 {'country': 'Brazil',
  'countryCode': 'BR',
  'name': 'Belo Horizonte',
  'parentid': 23424768,
  'placeType': {'code': 7, 'name': 'Town'},
  'url': 'http://where.yahooapis.com/v1/place/455821',
  'woeid': 455821},
 {'country': 'Brazil',
  'countryCode': 'BR',
  'name': 'Curitiba',
  'parentid': 23424768,
  'placeType': {'code': 7, 'name': 'Town'},
  'url': 'http://where.yahooapis.com/v1/place/455822',
  'woeid': 455822},
 {'country': 'Brazil',
  'countryCode': 'BR',
  'name': 'Porto Alegre',
  'parentid': 23424768,
  'placeType': {'code': 7, 'name': 'Town'},
  'url': 'htt

In [20]:
#Select the id desired from the previous step
# {'country': 'Brazil',
#   'countryCode': 'BR',
#   'name': 'Brazil',
#   'parentid': 1,
#   'placeType': {'code': 12, 'name': 'Country'},
#   'url': 'http://where.yahooapis.com/v1/place/23424768',
#   'woeid': 23424768}] <---
woeid = 23424768

In [167]:
#Get the trends
trends = api.request('trends/place',{'id':woeid}).json()

In [168]:
trends

[{'as_of': '2021-09-15T13:59:14Z',
  'created_at': '2021-09-14T12:26:13Z',
  'locations': [{'name': 'Brazil', 'woeid': 23424768}],
  'trends': [{'name': '#BolsonaroLadrão',
    'promoted_content': None,
    'query': '%23BolsonaroLadr%C3%A3o',
    'tweet_volume': None,
    'url': 'http://twitter.com/search?q=%23BolsonaroLadr%C3%A3o'},
   {'name': 'Baekhyun',
    'promoted_content': None,
    'query': 'Baekhyun',
    'tweet_volume': 199601,
    'url': 'http://twitter.com/search?q=Baekhyun'},
   {'name': 'Júpiter',
    'promoted_content': None,
    'query': 'J%C3%BApiter',
    'tweet_volume': 30070,
    'url': 'http://twitter.com/search?q=J%C3%BApiter'},
   {'name': 'ANGEL22 IS COMING',
    'promoted_content': None,
    'query': '%22ANGEL22+IS+COMING%22',
    'tweet_volume': None,
    'url': 'http://twitter.com/search?q=%22ANGEL22+IS+COMING%22'},
   {'name': '#diadocliente',
    'promoted_content': None,
    'query': '%23diadocliente',
    'tweet_volume': None,
    'url': 'http://twitter.

In [206]:
#Create the base graph
base = nx.Graph()

In [205]:
#Create lists to track the progress
search_endpoint = 'https://api.twitter.com/2/tweets/search/recent'
repeat = 1
to_search = [i['name'] for i in trends[0]['trends']]
searched = []

In [207]:
to_search

['#BolsonaroLadrão',
 'Baekhyun',
 'Júpiter',
 'ANGEL22 IS COMING',
 '#diadocliente',
 '#jogodepanelas',
 '#Gremio118Anos',
 'Marconny',
 '#redacaosportv',
 'Schumacher',
 'Luiza Trajano',
 'Malafaia',
 'MAMAMOO THE BEST ALBUM',
 'Surfin',
 'UM ANO DA TROPA',
 'R$1,99',
 'Ayres Brito',
 'youngjae',
 'Deselegante',
 'Parabéns Grêmio',
 'LOUDER THAN BOMBS',
 'Dores',
 'HOJE TEM FLAMENGO',
 'Sarí Corte Real',
 'Tardelli',
 'bighit',
 'Omar Aziz',
 'Killmonger',
 'Mirtes',
 'Lava Jato',
 'Copa do Brasil',
 'Crespo',
 'Noblat',
 'Mbappe',
 'Nem 10h',
 'Precisa Medicamentos',
 '15 de Setembro',
 'Dallagnol',
 'Liverpool x Milan',
 'Gorou',
 'Maracanã',
 'felix',
 'Banco Central',
 'addie larue',
 'André Mendonça',
 'Dia 15',
 'Léo Pinheiro',
 'Alcolumbre',
 'sula',
 'Kokomi']

In [222]:
#Get rate limiting values
rate_limiting = api.request('application/rate_limit_status').json()
remaining = rate_limiting['resources']['search']['/search/tweets']['remaining']
print(rate_limiting['resources']['search']['/search/tweets'])

#Loop while we can
while remaining>0:
  print(remaining)
  remaining-=repeat
  search = to_search.pop(0)
  searched.append(search)
  
  base.add_node(search)
  first = True
  max_id = float('Inf')
  result = api.request('search/tweets',params={'q':search,'count':100,'include_entities':True}).json()
  
  for _ in range(min(repeat,remaining)):
    if first:
      first = False
      result = api.request('search/tweets',params={'q':search,'count':100,'include_entities':True}).json()
    else:
      result = api.request('search/tweets',params={'q':search,'count':100,'max_id':max_id,'include_entities':True}).json()

    for r in result['statuses']:
      hash_len = len(r['entities']['hashtags'])
      if hash_len>=2:
        for i in range(hash_len):
          u = r['entities']['hashtags'][i]['text']
          _u = '#%s'%u
          if not _u in to_search and not _u in searched:
            to_search.append(_u)
          for j in range(i,hash_len):
            if i==j:
              continue
            v = r['entities']['hashtags'][j]['text']
            if base.has_edge(u,v):
              base.edges[u,v]['weight']+=1
            else:
              base.add_edge(u,v,weight=1)
      max_id = min(r['id'],max_id)

  time.sleep(0.5)

{'limit': 450, 'remaining': 0, 'reset': 1631716960}


In [223]:
G = base.copy()
nx.write_graphml(G,'g.graphml')
with open('searched.txt','w') as f:
  for i in searched:
    f.write('%s\n'%i)
with open('to_search.txt','w') as f:
  for i in to_search:
    f.write('%s\n'%i)

In [216]:
base.edges

EdgeView([('bolsonaroladrao', 'ForaBolsonaro'), ('bolsonaroladrao', 'ArthurLiraCumplice'), ('bolsonaroladrao', 'BolsonaroArregou'), ('bolsonaroladrao', 'BolsonaroTeEnganou'), ('bolsonaroladrao', 'forabolsonaro'), ('bolsonaroladrao', 'BolsonaroLadrão'), ('bolsonaroladrao', 'BolsonaroGenocida'), ('bolsonaroladrao', 'bolsonaroladrao'), ('bolsonaroladrao', 'ForaBolsonaroeSuaQuadrilha'), ('bolsonaroladrao', 'SilasMalafaia'), ('bolsonaroladrao', 'miliciano'), ('bolsonaroladrao', 'Fora'), ('bolsonaroladrao', 'Temer'), ('bolsonaroladrao', 'Lula'), ('bolsonaroladrao', 'AlexandreDeMoraes'), ('bolsonaroladrao', 'LulaInocente'), ('bolsonaroladrao', 'LulaHonesto'), ('bolsonaroladrao', 'genocidio'), ('bolsonaroladrao', 'EXCLUSIVA'), ('bolsonaroladrao', 'AmoedoNoFlow'), ('bolsonaroladrao', 'ForaBolsonaroGenocida'), ('bolsonaroladrao', 'impeachmentDeBolsonaro'), ('bolsonaroladrao', 'impeachmentJa'), ('bolsonaroladrao', 'VacinasSalvamVidas'), ('bolsonaroladrao', 'VacinaSim'), ('bolsonaroladrao', 'forag